In [2]:
import pandas as pd
from skportfolio import *
from skportfolio.datasets import *

In [251]:
def transaction_costs(
    old_portfolio: pd.Series,
    new_portfolio: pd.Series,
    buy_costs: float = 0.01,  # buying 1 share costs you 1% of that share
    sell_costs: float = 0.01,  # selling 1 share costs you 1% of that share
):
    """
    Calculates the incurred transaction costs for transfering between a portfolio
    with old_portfolios [$] and new_portfolio [$].

    Parameters
    ----------
    old_portfolio: pd.Series
    new_portfolio: pd.Series
    buy_costs: float
    sell_costs: float

    Returns
    -------
    Transaction costs
    """
    capital_allocation_difference = new_portfolio - old_portfolio
    capital_to_buy = (
        (capital_allocation_difference * (capital_allocation_difference > 0))
        .abs()
        .sum()
    )
    capital_to_sell = (
        (capital_allocation_difference * (capital_allocation_difference < 0))
        .abs()
        .sum()
    )
    return capital_to_buy * buy_costs + capital_to_sell * sell_costs

In [3]:
prices = load_dataset('tech_stocks')
prices.index = pd.to_datetime(prices.index)

In [4]:
triggers = pd.date_range(
    start = prices.index.min(),
    end = prices.index.max(),
    freq = "M"
)

In [6]:
freq = "M"

In [63]:
(prices.index == prices.index.to_period('W-MON'))

array([False, False, False, ..., False, False, False])

In [72]:
pd.Period(1,freq='M')

ValueError: Given date string not likely a datetime.

In [75]:
x=pd.DatetimeIndex(['2020-01-10'])
x in pd.Period(value=x[0], freq='M')

TypeError: argument of type 'Period' is not iterable

In [91]:
from IPython.display import display

In [238]:
# Here we should implement a backtester that takes one or more portfolio estimator objects,
# possibly a rebalancing policy, transaction costs
from typing import Union, Tuple, TypeVar, Sequence
import pandas as pd
from sklearn.base import BaseEstimator, MetaEstimatorMixin
from skportfolio import PortfolioEstimator


class Strategy(BaseEstimator, MetaEstimatorMixin):
    def __init__(
        self,
        initial_weights: pd.Series,
        initial_portfolio_value: float,
        estimator: PortfolioEstimator,
        rebalance_frequency: Union[int, str],
        lookback_periods: Union[int, pd.offsets.BaseOffset],
        turnover: pd.DatetimeIndex,
        transaction_costs: Union[float, Tuple[float, float]],
    ) -> None:
        self.initial_weights = initial_weights
        self.initial_portfolio_value = initial_portfolio_value
        self.estimator = estimator
        self.rebalance_frequency = rebalance_frequency
        self.lookback_periods = lookback_periods
        self.turnover = turnover
        self.transaction_costs = transaction_costs
        self.portfolio_values = [self.initial_portfolio_value]
        self.total_costs = []

    def fit(self, X, y, **kwargs):
        return self

In [273]:
strategy = Strategy(
    initial_weights=EquallyWeighted().fit(prices).weights_,
    initial_portfolio_value=1000,
    estimator=InverseVariance(),
    rebalance_frequency='6M',
    lookback_periods=60,
    turnover=0,
    transaction_costs=[0.01, 0.01] # buy and sell costs
)

In [274]:
prices = prices.resample('D').agg('first')

In [283]:
prices.iloc[0].dot(EquallyWeighted().fit(prices).weights_)

211.14869651794436

In [282]:
class Backtester(BaseEstimator):
    def __init__(self, strategy: Union[Strategy, Sequence[Strategy]]):
        self.strategy = strategy

    def fit(self, X, y=None, **kwargs):
        idx_freq = X.index.freq
        idx_freqstr = X.index.freqstr

        if idx_freqstr is None:
            raise IndexError("Please resample your data to given frequency")

        triggers = pd.date_range(start=X.index.min(), end=X.index.max(), freq="M")

        self.weights_ = pd.DataFrame(index=X.index, columns=X.columns, data=[])

        self.weights_.loc[
            (X.index.min() + pd.offsets.Day(self.strategy.lookback_periods)), :
        ] = self.strategy.initial_weights

        old_portfolio = self.strategy.initial_weights
        for t in X.index[self.strategy.lookback_periods :]:
            df_win = X.loc[(t - (self.strategy.lookback_periods * idx_freq)) : t, :]
            new_portfolio = self.strategy.estimator.fit(df_win).weights_
            self.weights_.loc[t, :] = new_portfolio
            last_prices = df_win.iloc[-1, :]
            self.portfolio_values.append(last_prices.dot(new_portfolio))
            trigger = np.any(df_win.index.max() == triggers)
            if trigger:
                trx_cost = transaction_costs(
                    old_portfolio=old_portfolio,
                    new_portfolio=new_portfolio
                )
                self.strategy.portfolio_values.append(
                    df_win.dot(new_portfolio).iloc[0] - trx_cost
                )
        return self

In [280]:
Backtester(strategy=strategy).fit(prices).weights_

,AAPL,MSTR,TSLA,MSFT,AMZN
date,,,,,
2016-08-22,NaN,NaN,NaN,NaN,NaN
2016-08-23,NaN,NaN,NaN,NaN,NaN
2016-08-24,NaN,NaN,NaN,NaN,NaN
2016-08-25,NaN,NaN,NaN,NaN,NaN
2016-08-26,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2021-08-16,0.258615,0.008723,0.078174,0.530608,0.12388
2021-08-17,0.262939,0.008623,0.074539,0.532704,0.121195
2021-08-18,0.242747,0.009426,0.072508,0.551329,0.12399


In [281]:
strategy.portfolio_values

[1000,
 989.3863349723816,
 989.3863349723816,
 978.3335303115845,
 967.7905400276184,
 956.989510154724,
 945.2444750785827,
 933.2869598960875,
 933.2869598960875,
 920.1949499702453,
 906.6213598823547,
 906.6213598823547,
 893.5665948295592,
 880.6487697792052,
 880.6487697792052,
 866.3853842735289,
 866.3853842735289,
 866.3853842735289,
 848.5969197654722,
 830.4058602714537,
 830.4058602714537,
 811.3124700736997,
 791.5357296943662,
 770.9734393501279,
 749.5204089546202,
 749.5204089546202,
 725.7764591026304,
 705.5715488243101,
 705.5715488243101,
 705.5715488243101,
 685.5090832710264,
 665.2320784187315,
 643.3801190757749,
 620.5915988731382,
 620.5915988731382,
 597.5317732810972,
 575.2643988990782,
 575.2643988990782,
 554.0329436302183,
 531.8882741737364,
 509.7558343696592,
 486.71334970474226,
 461.3386350440977,
 461.3386350440977,
 438.0512405586241,
 410.1039250373839,
 379.74839967727644,
 344.29170434951766,
 344.29170434951766,
 299.4573032569884,
 257.99100

In [ ]:
R=prices.rolling(4)
R.

In [ ]:
X.rolling(window = 10).apply(rebalance,)

In [ ]:
prices.index = pd.to_datetime(prices.index)

In [ ]:
import cvxpy as cp
import numpy as np

In [ ]:
model = MinimumVolatility()
def rebalance(df_prices):
    return model.fit(df_prices).weights_

In [ ]:
prices.resample('Y').apply(rebalance)